In [3]:
## QICK LIBRARY
from qick import *

from pynq.buffer import allocate

## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger, LFSR


import matplotlib.pyplot as plt
import numpy as np
import time

import qick.tprocv2_assembler as qasm
from importlib import reload  # Python 3.4+
    
##### Load FGPA BitStream
soc = QickSoc('./qick_216_spin_qtt.bit')


resetting clocks: 245.76 491.52
Not all DAC and ADC PLLs are locked. You may want to repeat the initialization of the QickSoc.


In [4]:
#soc.config_clocks(1)
print(soc)
#print(soc.qtt)

ZCU216
ZCU216
QICK running on ZCU216, software version 0.2.273

Firmware configuration (built Fri Aug  2 13:27:25 2024):

	Global clocks (MHz): tProcessor 291.840, RF reference 245.760

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (14.035 us)
		fs=4669.440 MHz, fabric=291.840 MHz, 32-bit DDS, range=4669.440 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (14.035 us)
		fs=4669.440 MHz, fabric=291.840 MHz, 32-bit DDS, range=4669.440 MHz
		DAC tile 1, blk 0 is 0_229, on JHC1

	0 readout channels:

	16 digital output pins:
	0:	PMOD0_0
	1:	PMOD0_1
	2:	PMOD0_2
	3:	PMOD0_3
	4:	PMOD0_4
	5:	PMOD0_5
	6:	PMOD0_6
	7:	PMOD0_7
	8:	PMOD1_0
	9:	PMOD1_1
	10:	PMOD1_2
	11:	PMOD1_3
	12:	PMOD1_4
	13:	PMOD1_5
	14:	PMOD1_6
	15:	PMOD1_7

	tProc qick_processor ("v2") rev 21: program memory 1024 words, data memory 1024 words
		external start pin: None


In [5]:
def qick_program (freq):
    reload(qasm)
    freq_int      = soc.freq2reg(freq)
    #period_cycles = soc.us2cycles(100/freq)
    period_cycles = soc.us2cycles(1000/freq)
    if (period_cycles > 65535):
        print('Waveform Longer than 16 bits > ', period_cycles)
    asm = f"""
    // set up pulse parameters
    //REG_WR w_freq imm #21845333
    REG_WR w_freq imm #{freq_int}
    //REG_WR w_phase imm #2000000000
      REG_WR w_phase imm #1500000000
    REG_WR w_gain imm #40000
    //REG_WR w_length imm #2000
    REG_WR w_length imm #{period_cycles}

    //REG_WR w_conf   imm  #1  // NoPHRST - NoPeriodic - DDS
    //REG_WR w_conf   imm  #5  // NoPHRST - Periodic   - DDS
    REG_WR w_conf   imm  #17 // PHRST   - NoPeriodic - DDS
    //REG_WR w_conf   imm  #21 // PHRST   - Periodic   - DDS

    REG_WR out_usr_time imm #500

    // play RF pulse
    WPORT_WR p0 r_wave
    WPORT_WR p1 r_wave

    REG_WR out_usr_time op -op(out_usr_time + #57)

    TRIG set p0
    TRIG set p1
    TRIG set p2

    REG_WR out_usr_time op -op(out_usr_time + w_length)

    TRIG clr p0
    TRIG clr p1
    TRIG clr p2

    TRIG set p3
    REG_WR out_usr_time op -op(out_usr_time + #10)
    TRIG clr p3


    JUMP HERE
    """
    p_txt, p_bin             = Assembler.str_asm2bin(asm)
    soc.tproc.Load_PMEM(p_bin)

def process(cfg_filter, cfg_slope, cfg_inter, cfg_smp=15):
    soc.qtt.set_config(cfg_filter, cfg_slope, cfg_inter, cfg_smp, 0)
    soc.qtt.arm()
    soc.tproc.start()
    time.sleep(.5)
    soc.tproc.stop()
    soc.qtt.disarm()
    tag_qty = soc.qtt.tag0_qty
  
    if (tag_qty > 2):
        tags = soc.qtt.read_mem('TAG0')
        adc_sr_int_period = adc_sr_period / pow(2, cfg_inter)
        tag_dif = tags[2:] - tags[1:-1]
        mean_period = int(np.mean(tag_dif) * adc_sr_int_period * 100 ) /100
        f = int(1000000/mean_period)/1000
        error = int(abs(f-freq)/f*100000)/1000  if (f != 0)  else 100
        print('Smp:', len(tags),' filt:', cfg_filter,'int:', cfg_inter,' Freq_meas', f, ' Freq_out',freq, 'error',error)   
        return tags
    else:
        print('tag0qty = ',tag_qty)
        return []

In [31]:
freq = 10.000
#freq = 12.485
#freq = 1.123
threshold  = 5000

qick_program(freq)
period        = 1000 / freq
dead_time     = (period/2)
adc_freq      = soc['adcs']['20']['f_fabric']
adc_period    = 1000 / adc_freq
adc_sr_period = adc_period / 8

# Set Threshold
soc.qtt.set_threshold(threshold)
# Set Dead Time
inhibit_clocks = int(dead_time / adc_period) + 1
inhibit_clocks = max(5, inhibit_clocks )
inhibit_clocks = min(255, inhibit_clocks )
soc.qtt.set_dead_time(inhibit_clocks)
print('inhibit_clocks',inhibit_clocks)
#soc.qtt.print_axi_regs()

##########################################################################
#process(cfg_filter, cfg_slope, cfg_inter)
#soc.qtt.print_axi_regs()
tags = process(0, 0, 0)
tags = process(0, 0, 1)
tags = process(0, 0, 2)
tags = process(0, 0, 3)
tags = process(0, 0, 4)
tags = process(1, 0, 0)
tags = process(1, 0, 1)
tags = process(1, 0, 2)
tags = process(1, 0, 3)
tags = process(1, 0, 4)


inhibit_clocks 15
Smp: 1007  filt: 0 int: 0  Freq_meas 10.031  Freq_out 10.0 error 0.309
Smp: 1007  filt: 0 int: 1  Freq_meas 10.031  Freq_out 10.0 error 0.309
Smp: 1007  filt: 0 int: 2  Freq_meas 10.031  Freq_out 10.0 error 0.309
Smp: 1007  filt: 0 int: 3  Freq_meas 10.031  Freq_out 10.0 error 0.309
Smp: 1007  filt: 0 int: 4  Freq_meas 10.031  Freq_out 10.0 error 0.309
Smp: 1007  filt: 1 int: 0  Freq_meas 10.031  Freq_out 10.0 error 0.309
Smp: 1007  filt: 1 int: 1  Freq_meas 10.031  Freq_out 10.0 error 0.309
Smp: 1007  filt: 1 int: 2  Freq_meas 10.031  Freq_out 10.0 error 0.309
Smp: 1007  filt: 1 int: 3  Freq_meas 10.031  Freq_out 10.0 error 0.309
Smp: 1007  filt: 1 int: 4  Freq_meas 10.031  Freq_out 10.0 error 0.309


In [ ]:
soc.qtt.print_axi_regs()

smp = soc.qtt.smp_qty
print('SMP_QTY',smp)
arm = soc.qtt.arm_qty
print('ARM_QTY',arm)
arms = soc.qtt.read_mem('TAG0')
arms = soc.qtt.read_mem('TAG1')
arms = soc.qtt.read_mem('TAG2')
arms = soc.qtt.read_mem('TAG3')
arms = soc.qtt.read_mem('ARM')
print(arms)
arms = soc.qtt.read_mem('SMP')


#smp = soc.qtt.read_mem('SMP').astype(int)
#plt.plot(smp)

